In [ ]:
###infosite: https://min-api.cryptocompare.com/documentation?key=Blockchain&cat=blockchainListOfCoins&api_key=29fa56d048dcb48593c544e6e5cf002fc4136ef00acf711cb12eccc19801f9ad

In [2]:
import urllib.request
import pandas as pd
from datetime import datetime

api_key = '29fa56d048dcb48593c544e6e5cf002fc4136ef00acf711cb12eccc19801f9ad'

### Daily Pair OHLCV

In [65]:
apicall_coins = ['BTC','ETH']
apicall_limit = '100'
# make empty data frame 
df = pd.DataFrame(columns= ['close', 'conversionSymbol','conversionType','high','low','open','volumefrom','volumeto','date'])

for i in range(0,len(apicall_coins)):
    url = 'https://min-api.cryptocompare.com/data/v2/histoday?fsym=' + str(apicall_coins[i]) + '&tsym=USD&limit=' + apicall_limit + '&api_key=' + api_key
    print('url request: ' + url)
    raw_data2 = pd.read_json(urllib.request.urlopen(url).read())
    historicaldata1 = raw_data2['Data'].iloc[3]
    historicaldata1 = pd.DataFrame(historicaldata1)
#add to data frame instead of making a new data frame
    historicaldata1['conversionSymbol'] = str(apicall_coins[i])
    historicaldata1['time'] = pd.to_datetime(historicaldata1['time'],unit='s')
    historicaldata1['date'] = historicaldata1['time']
    historicaldata1 = historicaldata1.drop('time',axis=1)
    df = pd.concat([df, historicaldata1])

url request: https://min-api.cryptocompare.com/data/v2/histoday?fsym=BTC&tsym=USD&limit=100&api_key=29fa56d048dcb48593c544e6e5cf002fc4136ef00acf711cb12eccc19801f9ad
url request: https://min-api.cryptocompare.com/data/v2/histoday?fsym=ETH&tsym=USD&limit=100&api_key=29fa56d048dcb48593c544e6e5cf002fc4136ef00acf711cb12eccc19801f9ad


In [61]:
df.describe()

,close,high,low,open,volumefrom,volumeto
count,202.000000,202.000000,202.000000,202.000000,2.020000e+02,2.020000e+02
mean,4249.871535,4351.637723,4151.662277,4261.461040,1.606187e+06,4.592226e+08
std,4171.468547,4259.617788,4080.859000,4177.249802,1.917531e+06,3.400214e+08
min,110.230000,120.810000,105.220000,110.230000,1.877830e+03,1.396761e+06
25%,167.105000,170.437500,164.410000,167.105000,3.039829e+04,2.335680e+08
50%,2601.210000,2827.465000,2111.925000,2601.210000,3.027800e+05,3.532549e+08
75%,8517.032500,8723.252500,8284.090000,8517.032500,2.846494e+06,5.611755e+08
max,10360.370000,10500.540000,10250.550000,10360.370000,1.098033e+07,1.769477e+09


### Iterator

In [152]:
def api_iterator(api_set_limit,earliestdate):
    earliestdate = pd.to_datetime(earliestdate)
    numofdays = abs(datetime.today()- earliestdate)
    numofdays = str(numofdays)
    numofdays = numofdays[0:3]
    iterations =  int(numofdays)/api_set_limit
    return iterations 



### webscaper 

In [17]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time

source = requests.get('https://www.coindesk.com').text

soup = BeautifulSoup(source,'lxml')

###headline section###

headlinesection = soup.find('section',class_='right-column')
headlineauthorcards = headlinesection.find('div',class_='card-desc')

##print(topsection.prettify())

##dataframe##
ts = time.strftime("%Y-%m-%d %H:%M:%S", time.gmtime())
cdeskarticles = pd.DataFrame()

##empty lists for df##
headlines = []
headlinesummaries = []
headlinesauthors = []
headlinespubdates = []

##loops to fill df##
for article in headlinesection.find_all('div',class_ = 'card-text-block'):
    headline = article.h2.a.text
    headlines.append(headline)

for summary in headlinesection.find_all('p'):
    headlinesummary = summary.text
    headlinesummaries.append(headlinesummary) 

for pubdate in headlinesection.find_all('span', class_= 'card-date'):
    headlinespubdate = pubdate.text
    headlinespubdates.append(headlinespubdate)



##insert into df##
cdeskarticles['title'] = headlines
cdeskarticles['summary'] = headlinesummaries
cdeskarticles['pub_date'] = headlinespubdates
cdeskarticles['placement'] = 'headlines'
cdeskarticles['imported_timestamp'] = ts
cdeskarticles = cdeskarticles[['imported_timestamp','pub_date','placement','title','summary']]
cdeskarticles

,imported_timestamp,pub_date,placement,title,summary
0,2020-03-24 03:15:06,"Mar 23, 2020",headlines,Ruble’s Drop Eases Pain of Lower Bitcoin Price...,The global market meltdown has indirectly bene...
1,2020-03-24 03:15:06,"Mar 23, 2020",headlines,"HODLpac Enlists Winklevoss Twins, Brian Armstr...",A new crypto-focused political action committe...
2,2020-03-24 03:15:06,"Mar 23, 2020",headlines,"Bitcoin, Gold Spike as Fed Unveils Unlimited C...",The Federal Reserve has announced a quantitati...
3,2020-03-24 03:15:06,"Mar 23, 2020",headlines,Privacy Advocates Are Sounding Alarms Over Cor...,Crises provide opportunities for surveillance ...
4,2020-03-24 03:15:06,"Mar 23, 2020",headlines,Long-Festering DeFi Dapp Bug Still Not Fixed b...,ZenGo is renewing a debate over personal respo...
5,2020-03-24 03:15:06,"Feb 28, 2020",headlines,How to Contact CoinDesk’s Editors and Reporter...,We've made it a lot easier to reach us. Our ne...


In [18]:
###nonheadline section###
nonheadlinesection = soup.find('section',class_='list-body')

##dataframe##
ts = time.strftime("%Y-%m-%d %H:%M:%S", time.gmtime())
cdesksubarticles = pd.DataFrame()

##empty lists for df##
subheadlines = []
subsummaries = []
subpubdates = []

##print(nonheadlinesection.prettify())

for article in nonheadlinesection.find_all('h4'):
    subheadline = article.text
    subheadlines.append(subheadline)

for sumtext in nonheadlinesection.find_all('p',class_ = 'card-text'):
    subsummary = sumtext.text
    subsummaries.append(subsummary)
    
for pubdate in nonheadlinesection.find_all('div',class_ = 'card-desc-block'):
    subpubdate = pubdate.time.text
    subpubdates.append(subpubdate)

cdesksubarticles['title'] = subheadlines
cdesksubarticles['summary'] = subsummaries
cdesksubarticles['pub_date'] = subpubdates
cdesksubarticles['placement'] = 'subarticles'
cdesksubarticles['imported_timestamp'] = ts
cdesksubarticles = cdesksubarticles[['imported_timestamp','pub_date','placement','title','summary']]
cdeskarticles = pd.concat([cdeskarticles,cdesksubarticles])
cdeskarticles.reset_index(drop=True)

,imported_timestamp,pub_date,placement,title,summary
0,2020-03-24 03:15:06,"Mar 23, 2020",headlines,Ruble’s Drop Eases Pain of Lower Bitcoin Price...,The global market meltdown has indirectly bene...
1,2020-03-24 03:15:06,"Mar 23, 2020",headlines,"HODLpac Enlists Winklevoss Twins, Brian Armstr...",A new crypto-focused political action committe...
2,2020-03-24 03:15:06,"Mar 23, 2020",headlines,"Bitcoin, Gold Spike as Fed Unveils Unlimited C...",The Federal Reserve has announced a quantitati...
3,2020-03-24 03:15:06,"Mar 23, 2020",headlines,Privacy Advocates Are Sounding Alarms Over Cor...,Crises provide opportunities for surveillance ...
4,2020-03-24 03:15:06,"Mar 23, 2020",headlines,Long-Festering DeFi Dapp Bug Still Not Fixed b...,ZenGo is renewing a debate over personal respo...
5,2020-03-24 03:15:06,"Feb 28, 2020",headlines,How to Contact CoinDesk’s Editors and Reporter...,We've made it a lot easier to reach us. Our ne...
6,2020-03-24 03:15:10,"Mar 23, 2020",subarticles,Bitcoin and Gold: Evaluating Hard-Cap Currenci...,How would a bitcoin economy react to coronavir...
7,2020-03-24 03:15:10,"Mar 23, 2020",subarticles,"Investors Look to Gold, Crypto After Fed Goes ...",Gold is up Monday and so are most cryptocurren...
8,2020-03-24 03:15:10,"Mar 23, 2020",subarticles,Unlimited QE and Why Markets Can’t Price In CO...,The Market reacts as the Fed announced effecti...
9,2020-03-24 03:15:10,"Mar 23, 2020",subarticles,Bitcoin: A Global Port in a Market Storm?,Turmoil in global markets and economies points...


In [ ]:
##headline WIP##
for author in headlinesection.find_all('span', class_= 'card-author'):
    for author2 in author
    headlinesauthor = author.a.text
    print(headlinesauthor)
    ##headlineauthors.append(headlinesauthor)
    print(headlinesauthors)
    
##for link in headlinesection.find_all('h2',class_ = ''):
    ##headlinelink = article.h2.a.link
    ##headlinelinks.append(headlinelink)
    ##print(headlinelinks)

overall = []
for i in desc:
    hld = []
    for j in authors:
        hld.append(j)
    
    overall.append(hld)

### Available List

In [83]:
url = 'https://min-api.cryptocompare.com/data/blockchain/list?api_key=' + api_key
raw_data4 = pd.read_json(urllib.request.urlopen(url))
raw_data4

,Data,HasWarning,Message,RateLimit,Response,Type
0XBTC,"{'id': 877383, 'symbol': '0XBTC', 'partner_sym...",False,,NaN,Success,100
1ST,"{'id': 28328, 'symbol': '1ST', 'partner_symbol...",False,,NaN,Success,100
1WO,"{'id': 925200, 'symbol': '1WO', 'partner_symbo...",False,,NaN,Success,100
AAC,"{'id': 925291, 'symbol': 'AAC', 'partner_symbo...",False,,NaN,Success,100
ABCC,"{'id': 925864, 'symbol': 'ABCC', 'partner_symb...",False,,NaN,Success,100
ABT,"{'id': 758055, 'symbol': 'ABT', 'partner_symbo...",False,,NaN,Success,100
ABYSS,"{'id': 419711, 'symbol': 'ABYSS', 'partner_sym...",False,,NaN,Success,100
ACCN,"{'id': 716854, 'symbol': 'ACCN', 'partner_symb...",False,,NaN,Success,100
ACE,"{'id': 294364, 'symbol': 'ACE', 'partner_symbo...",False,,NaN,Success,100
ADA,"{'id': 321992, 'symbol': 'ADA', 'partner_symbo...",False,,NaN,Success,100
